You can either download the pitchfork data directly or you can use the following command to do.

The file is too big to add to Github. Just make sure it's in the data directory

In [1]:
!wget https://www.dropbox.com/s/wp8qr1elowcf3lm/pitchfork.db 

--2020-10-30 14:32:57--  https://www.dropbox.com/s/wp8qr1elowcf3lm/pitchfork.db
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wp8qr1elowcf3lm/pitchfork.db [following]
--2020-10-30 14:32:57--  https://www.dropbox.com/s/raw/wp8qr1elowcf3lm/pitchfork.db
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb0efd6c9e89938a332b4316359.dl.dropboxusercontent.com/cd/0/inline/BCTwOJ4eHQOyqpFptfSSs5Zhl0z0huFazSHnaenFMIeliKP-pbsuPuolADWkUgy_q6Au-6mOAMlAX2iuNkpmJkgZT1coD04ugF4PHKqoi3BvJw/file# [following]
--2020-10-30 14:32:57--  https://ucb0efd6c9e89938a332b4316359.dl.dropboxusercontent.com/cd/0/inline/BCTwOJ4eHQOyqpFptfSSs5Zhl0z0huFazSHnaenFMIeliKP-pbsuPuolADWkUgy_q6Au-6mOAMlAX2iuNkpmJkgZT1coD04ugF4PHKqoi3BvJw/file
Resolving ucb0efd6c9e89938a332b4316359.dl.drop

In [5]:
# Moves pitchfork db file if you used wget to download it
import shutil
shutil.move("pitchfork.db", "Data/pitchfork.db")

'Data/pitchfork.db'

Pitchfork data is a SQL database, so we use sqlite3 to load and query it

In [8]:
p4k_file = "data/pitchfork.db"

In [13]:
#Install if you don't have it already 
#!pip3 install sqlite3

In [18]:
import sqlite3
import pandas as pd

In [19]:
con = sqlite3.connect(p4k_file)

In [25]:
df = pd.read_sql_query("SELECT * FROM reviews", con = con, index_col=["id"])
df.head()

,artist,album,genre,score,date,author,role,review,bnm,link
id,,,,,,,,,,
0,David Byrne,“…The Best Live Show of All Time” — NME EP,Rock,5.5,January 11 2019,Andy Beta,Contributor,"Viva Brother, Terris, Mansun, the Twang, Joe L...",0,https://pitchfork.com/reviews/albums/david-byr...
1,DJ Healer,Lost Lovesongs / Lostsongs Vol. 2,Electronic,6.2,January 11 2019,Chal Ravens,Contributor,"The Prince of Denmark—that is, the proper prin...",0,https://pitchfork.com/reviews/albums/dj-healer...
2,Jorge Velez,Roman Birds,Electronic,7.9,January 10 2019,Philip Sherburne,Contributing Editor,"Jorge Velez has long been prolific, but that’s...",0,https://pitchfork.com/reviews/albums/jorge-vel...
3,Chandra,Transportation EPs,Rock,7.8,January 10 2019,Andy Beta,Contributor,When the Avalanches returned in 2016 after an ...,0,https://pitchfork.com/reviews/albums/chandra-t...
4,The Chainsmokers,Sick Boy,Electronic,3.1,January 9 2019,Larry Fitzmaurice,Contributor,We’re going to be stuck with the Chainsmokers ...,0,https://pitchfork.com/reviews/albums/the-chain...


You can create document term matrics from LIWC features

In [38]:
liwc={}
with open("liwc.dictionary.txt") as file:
    for line in file:
        cols=line.rstrip().split("\t")
        word=cols[0]
        liwcs=cols[1].split(" ")
        liwc[word]=liwcs

In [40]:
from nltk.tokenize import RegexpTokenizer 
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
tk = RegexpTokenizer('\s+', gaps = True)
def liwc_tokenizer(text):
    text = text.lower()
    tokens = tokenize(text)
    
    output = []
    for word in tokens:
        if word in liwc:
            for l in liwc[word]:
                output.append(l)
    return output

In [43]:
review_text = df.review
cv = CountVectorizer(analyzer=liwc_tokenizer)
dtm = cv.fit_transform(review_text)

In [48]:
liwc_dtm = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names())